In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [2]:
df_gastos = pd.read_csv("../datos/gastos.csv")
df_gastos.head(1)

,año,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,...,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto
0,2020,113.0,1535894,344330.0,169443,0.0,169443.0,1877.0,678201.0,342043.0,...,506018.0,26209836,19875799,349499.0,5984538,0.0,298707.0,-6504.0,50165.0,28810972


In [3]:
df_ingresos = pd.read_csv("../datos/ingresos.csv", index_col=0)
df_ingresos.head(1)

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,tipo,año,ncodi
0,0.0,103677.0,64383.0,39294.0,61456.0,Hospital,2013,"990,00"


In [4]:
df_ingresos.dtypes

particulares               float64
aseguradoras               float64
aseguradoras_enfermedad    float64
aseguradoreas_trafico      float64
mutuas                     float64
tipo                        object
año                          int64
ncodi                       object
dtype: object

Eliminamos las filas cuyo ncodi es nan y las ponemos de tipo int.

In [5]:
df_ingresos = df_ingresos[df_ingresos["ncodi"].notna()]

In [6]:
df_ingresos["ncodi"] = df_ingresos["ncodi"].str.split(",", expand=True)[0]
df_ingresos["ncodi"] = df_ingresos["ncodi"].str.split(".", expand=True)[0]
df_ingresos["ncodi"] = df_ingresos["ncodi"].astype(int)

In [7]:
df_ingresos["ncodi"].dtype

dtype('int64')

In [8]:
df_ingresos.dtypes

particulares               float64
aseguradoras               float64
aseguradoras_enfermedad    float64
aseguradoreas_trafico      float64
mutuas                     float64
tipo                        object
año                          int64
ncodi                        int64
dtype: object

Vamos a sustituir el tipo por su id:

In [9]:
df_tipo_hospi = pd.DataFrame(df_ingresos["tipo"].unique())
df_tipo_hospi.index += 1
df_tipo_hospi = df_tipo_hospi.reset_index().rename(columns={"index":"tipo_id", 0:"nombre"})

In [10]:
df_tipo_hospi.to_csv("../datos/df_tipo_hospitalizacion.csv")

Sustituimos:

In [11]:
df_ingresos = df_ingresos.merge(df_tipo_hospi, how="left", left_on="tipo", right_on="nombre").drop(columns=["tipo", "nombre"])
df_ingresos

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,año,ncodi,tipo_id
0,0.0,103677.0,64383.0,39294.0,61456.0,2013,990,1
1,324687.0,478280.0,392084.0,86196.0,484228.0,2013,680,1
2,793512.0,1803352.0,1803352.0,0.0,20402.0,2013,411,1
3,0.0,0.0,0.0,0.0,0.0,2013,241,1
4,24406.0,63557.0,55091.0,8466.0,44106.0,2013,558,1
...,...,...,...,...,...,...,...,...
55303,0.0,0.0,0.0,0.0,0.0,2016,970,6
55304,0.0,0.0,0.0,0.0,0.0,2016,1019,6
55305,0.0,0.0,0.0,0.0,0.0,2016,1016,6
55306,0.0,0.0,0.0,0.0,0.0,2016,1047,6


In [12]:
df_ingresos.columns

Index(['particulares', 'aseguradoras', 'aseguradoras_enfermedad',
       'aseguradoreas_trafico', 'mutuas', 'año', 'ncodi', 'tipo_id'],
      dtype='object')

Reordenamos los indices:

In [13]:
df_ingresos = df_ingresos.reindex(columns=['año', 'particulares', 'aseguradoras', 'aseguradoras_enfermedad',
       'aseguradoreas_trafico', 'mutuas', 'ncodi', 'tipo_id'])

In [14]:
df_ingresos.to_csv("../datos/df_ingresos_final.csv")